# Assignment 2: Practical Machine Learning Project Report



31005/32513 Machine Learning Spring 2019, Assignment 2



---



> Group members: Pit Wegner, Xianfeng Zhuge, Kailei Wu




## Introduction

Nowadays, more and more activity recognition-based machine learning has been studied, especially in human physical activity recognition. The progress in this direction is surprisingly good. Models can recognize simple human exercises, such as walking, running or standing, count steps and detect a general overarching activity. However, it is hard to apply in a specific domain such as healthcare. There are mainly for two reasons: the activities themselves are more complex, potentially involving other subjects, and additionally harder to analyze because of the lack of public data that can be used within the research. In this report, we create a machine learning (ML) model regarding the "Open Lab Nursing Activity Recognition Challenge", which asks to recognize six different activities within the nursing area (Lago et al., 2019).

In recent years, many activity recognition algorithms in health care applications are focusing on patients or doctors rather than nurses. However, the recognition of nursing activities can be helpful in the nursing domain, such as for automatic record creation and standardization of operation supervision. Until now, there is still a gap for ML models to recognize nursing-related activities due to the problems that we mentioned before. The applications of such a model would be wide-spread and of high impact in the nursing area, given the model achieves an exceptional accuracy in prediction.

In this report, a machine learning technique to tackle the "Open Lab Nursing Activity Recognition Challenge" is proposed and evaluated. First, we will explain the data provided, including data preprocessing and data structure design. Next, the CNN model implementation that solved the problem will be presented, followed by a test result performance analysis of the model and an evaluation for the whole experiment. Before concluding, methods of improvement and future work regarding the project are formulated. 


## Exploration

The “Open Lab Nursing Activity Recognition Challenge” provides several sets of data for training propose, we selected the data that including 

The data that we used for this experiment is 

1. from the paper
2. data from the challenge, we are using motion capture data (sigment) and activitiy label 
3. data preprocessing, data split, data sampling (random pick windows, no doulbe for each epoch) 
4. CNN model, 1 conv, 1 pool, 2 fully connected, activation function on conv and fully connected1 (with graph) lossing(cross entropy loss) optimizer (adam (adaptive moment extimation))

5. test: normal test round, then sigment voting 




![Architecture](https://github.com/pitwegner/UTS_ML2019_Project/blob/master/img/architecture.svg)

## Methodology 

training: 

1. input the data 
2. mini batch: size 32
3. learning rate 0.000001
4. forward pass, backward pass, optimize
5. at the end of each epoch, we did validation(see the validation loss)
6. stop learning if valodation loss is not decrease for 10 epoch

testing: 

1. forward pass for test data set.
2. conpare to the labels, create confusion matrix, calculate the accuracy. 
3. majority vote for siugments, select the mode, calculate the accuracy.




## Evaluation

### Model Performance

![Random_Segment_Split](https://github.com/pitwegner/UTS_ML2019_Project/blob/master/img/plot_random_segment_split.png)
Split randomly by segments

|     *    | $l_2$ | $l_3$ | $l_4$ | $l_6$ | $l_9$ | $l_{12}$ |
| -------- | ----- | ----- | ----- | ----- | ----- | -------- |
| $p_2$    |   366 |   103 |    68 |   185 |    61 |       39 |
| $p_3$    |    88 |   411 |    36 |     4 |     5 |        3 |
| $p_4$    |    83 |    80 |   181 |    49 |   210 |       62 |
| $p_6$    |    25 |    25 |     2 |   146 |    74 |       92 |
| $p_9$    |    39 |     4 |     6 |    32 |   269 |      103 |
| $p_{12}$ |   171 |    10 |    21 |    27 |   142 |      688 |
 
Accuracy: 52.71%

Entire segments:

| segment | 4 | 31 | 56 | 57 | 63 | 88 | 101 | 173 | 185 | 194 | 215 | 227 | 241 | 248 | 290 | 295 | 305 | 306 | 336 | 362 | 373 |
| - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| label | 2 | 4 | 12 | 2 | 2 | 2 | 6 | 9 | 9 | 12 | 3 | 9 | 2 | 3 | 4 | 12 | 3 | 12 | 9 | 3 | 9 |
| predict | 2 | 2 | 12 | 2 | 4 | 2 | 2 | 9 | 9 | 12 | 2 | 12 | 2 | 3 | 4 | 12 | 3 | 12 | 6 | 3 | 9 |
| **segment** | **385** | **480** | **508** | **515** | **531** | **559** | **573** | **607** | **620** | **631** | **639** | **640** | **667** | **689** | **703** | **711** | **748** | **753** | **765** | **777** | **795** |
| label | 3 | 2 | 6 | 9 | 6 | 12 | 3 | 6 | 3 | 12 | 12 | 12 | 12 | 2 | 9 | 12 | 2 | 4 | 9 | 12 | 12 |
| predict | 3 | 12 | 4 | 12 | 2 | 12 | 3 | 6 | 4 | 12 | 12 | 12 | 12 | 2 | 4 | 12 | 2 | 4 | 9 | 12 | 12 |

Accuracy 71.43%

![Person_Split](https://github.com/pitwegner/UTS_ML2019_Project/blob/master/img/plot_person_split.png)
Split by Person

|     *    | $l_2$ | $l_3$ | $l_4$ | $l_6$ | $l_9$ | $l_{12}$ |
| -------- | ----- | ----- | ----- | ----- | ----- | -------- |
| $p_2$    |   186 |   212 |   205 |   231 |    78 |      148 |
| $p_3$    |    60 |   305 |    34 |     0 |     3 |        0 |
| $p_4$    |    46 |    58 |   126 |    17 |    81 |      118 |
| $p_6$    |     7 |    18 |     0 |    45 |    33 |       49 |
| $p_9$    |    51 |   103 |   104 |    13 |   207 |      118 |
| $p_{12}$ |   371 |   437 |    78 |   122 |   138 |      451 |

Accuracy: 31.04%

Entire segments:

| segment | 0 | 56 | 67 | 85 | 99 | 138 | 143 | 144 | 145 | 189 | 196 | 200 | 210 | 213 | 219 | 227 | 229 | 243 | 283 | 305 | 328 |
| - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| label | 6 | 12 | 12 | 2 | 3 | 3 | 3 | 4 | 12 | 6 | 2 | 9 | 9 | 3 | 2 | 9 | 2 | 9 | 4 | 3 | 3 |
| predict | 2 | 12 | 12 | 2 | 3 | 12 | 2 | 2 | 12 | 2 | 12 | 12 | 12 | 12 | 12 | 9 | 2 | 9 | 2 | 2 | 12 |
| **segment** | **340** | **380** | **387** | **406** | **432** | **441** | **459** | **515** | **604** | **612** | **613** | **625** | **639** | **662** | **669** | **674** | **680** | **682** | **685** | **686** | **723** | **773** |
| label | 12 | 9 | 12 | 4 | 12 | 12 | 3 | 9 | 12 | 3 | 2 | 4 | 12 | 3 | 4 | 3 | 6 | 2 | 2 | 12 | 6 | 3 |
| predict | 12 | 2 | 12 | 2 | 12 | 12 | 12 | 9 | 12 | 2 | 2 | 9 | 12 | 3 | 4 | 3 | 2 | 12 | 12 | 4 | 2 | 12 |

Accuracy: 44.19%

## Conclusion

## Ethical Considerations

As a machine learning model, common ethical issues considering the application in a real world domain arise naturally. These typically aim at potential mistakes, training bias and data protection. Escpecially in the medical domain, mistakes can have a huge impact on a human life. In the case of our model, however, a wrong documentation entry about a nurse activity has a limited capability to directly cause damage. Patient health can be compromised by a left out or wrong procedure, for example developing pressure ulcers as a result of improper turning and repositioning of bedridden patients. The documentation record can then be used to trace back where the mistake has happened. Thus, our model provides support in preventing and retracing clinical incidents without directly impacting patient safety. Additionally, giving the nurse a chance to manually adjust the report after its generation further mitigates the impact of erroneous results. Other benefits of applying the model in nursing include a decreased workload and a more complete activity report for accounting purposes.

Activity recognition in a workplace also adds ethical complications. If not protected, tool could easily be (mis-)used by managers to monitor workforce activity, intruding into their privacy. The insights could result in different treatment for individual employees, even up to terminating the employment. Privacy impairment from the data pool itself is a further issue, since detailed movements are possibly visible from motion capture data, even if much harder to process than, say video. Thus, secure storage and access to data, as well as a proper authorization to the model results are crucial for an implementation.

Considering the results being biased toward training data, the risk of discrimination only leads to more inaccurate results for new employees. As this is known and can be tested beforehand, the impact is minimal. Overall, with proper data protection and sufficient accuracy, the benefits of the technique outweigh the possible risks. From a utalitarian point of view, the use of our model can hence be considered ethically acceptable. A deontological approach also does not object. Since results can be modified, the autonomy and freedom of will are not inhibited by applying the model, as long as privacy is ensured. Furthermore, the universalization of the application neither contradict with the intention nor with itself.

In [ ]:
d = {625: [9, 4], 138: [12, 3], 196: [12, 2], 613: [2, 2], 243: [9, 9], 387: [12, 12], 459: [12, 3], 0: [2, 6], 432: [12, 12], 99: [3, 3], 612: [2, 3], 305: [2, 3], 328: [12, 3], 143: [2, 3], 639: [12, 12], 144: [2, 4], 680: [2, 6], 515: [9, 9], 145: [12, 12], 685: [12, 2], 213: [12, 3], 674: [3, 3], 682: [12, 2], 56: [12, 12], 773: [12, 3], 604: [12, 12], 189: [2, 6], 723: [2, 6], 200: [12, 9], 85: [2, 2], 227: [9, 9], 669: [4, 4], 219: [12, 2], 380: [2, 9], 67: [12, 12], 406: [2, 4], 662: [3, 3], 210: [12, 9], 340: [12, 12], 283: [2, 4], 686: [4, 12], 441: [12, 12], 229: [2, 2]}
s = '| segment |'
for k in sorted(d.keys())[:len(d.keys())//2]:
    s += ' {} |'.format(k)
s += "\n| - |"
for k in sorted(d.keys())[:len(d.keys())//2]:
    s += ' - |'
s += "\n| label |"
for k in sorted(d.keys())[:len(d.keys())//2]:
    s += ' {} |'.format(d[k][1])
s += '\n| predict |'
for k in sorted(d.keys())[:len(d.keys())//2]:
    s += ' {} |'.format(d[k][0])
s += '\n| **segment** |'
for k in sorted(d.keys())[len(d.keys())//2:]:
    s += ' **{}** |'.format(k)
s += "\n| label |"
for k in sorted(d.keys())[len(d.keys())//2:]:
    s += ' {} |'.format(d[k][1])
s += '\n| predict |'
for k in sorted(d.keys())[len(d.keys())//2:]:
    s += ' {} |'.format(d[k][0])
print(s)